# Ifaz Chowdhury 

## Research question/interests

 ** I want to explore the relationship between xG90 and xA90, which basically helps assess the efficiency of a player in a 90 minute game by seeing how many goals and assists a player makes. This helps understand player contribution and player performance better. **